In [40]:
import torch
from torchvision import models
from torchvision import transforms
from PIL import Image
import numpy as np

In [2]:
model = models.segmentation.deeplabv3_resnet101(pretrained=True)

/Users/macbook/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/macbook/miniconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /Users/macbook/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth
100%|████████████████████████████████████████| 233M/233M [00:48<00:00, 5.06MB/s]


In [13]:
image = Image.open("example.jpeg")

In [16]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [17]:
input_tensor = preprocess(image).unsqueeze(0)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_batch = input_tensor.to(device)

In [30]:
model.eval()
with torch.inference_mode():

    output = model(input_batch)['out'][0]
output_predictions = output.argmax(0).cpu().numpy()

In [ ]:
# create mask with background
background_mask = np.where(output_predictions == 0, 0, 1)

# mask original image
foreground = np.array(image) * np.expand_dims(background_mask, axis=2)


In [47]:
background_image = Image.open('fon.jpeg')
background_image = background_image.resize(image.size)

In [49]:
background_array = np.array(background_image)

In [50]:
background = background_array * np.expand_dims(1 - background_mask, axis=2)

In [51]:
composite_image = foreground + background

In [53]:
result_image = Image.fromarray(composite_image.astype('uint8'))
result_image.show()
result_image.save('result.jpg')